In [ ]:
%pylab inline

sys.path.append('/home/peter/code/projects/')
sys.path.append('/home/ubuntu/')
from aidevutil import *
from massimport import *
from sklearn import datasets

import project_common, genetic, paramspace
from project_common import *
from paramspace import *
from itertools import product
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
from skopt.benchmarks import branin as branin
from skopt.benchmarks import hart6 as hart6_
from functools import partial
from skopt.plots import plot_evaluations, plot_convergence
from skopt.plots import plot_objective
from skopt import gp_minimize, forest_minimize, dummy_minimize
from skopt import callbacks
#from skopt.callbacks import CheckpointSaver
from skopt import load

In [ ]:
#if socket.gethostname() in ['laptop','desktop']:
#    %load_ext birdseye
#    %config BirdsEyeMagics.port = 7777
#    %config BirdsEyeMagics.bind_host = '192.168.1.149'

### General parameters

In [ ]:
parallel = 0
num_trials = 1
use_CV = 1
CV_splits = 5
test_size = 1/5
datasplit_mode = 'random'
use_preps = 0
use_hpo = 0
exhaustive_search_on = 1
genetic_search_on = 0
min_ensemble_size=1
max_ensemble_size=3

stack_original_level1 = False
optimized_voting = 1
voting_opt_trials = 10000

pdict = dict(num_trials=1, use_CV=use_CV, CV_splits=CV_splits)

min_population_size = 128
num_generations = 5
max_evaluations = 5000
generational_mode = 0
plot_nns = 0

nrows = 10000000
hpo_nrows = 10000000
use_hpo_skopt = 0

prediction_mode = 'binary_classification'

In [ ]:
titanic_version = 1

In [ ]:
genetic4.MinNumClassifiers = min_ensemble_size
genetic4.MaxNumClassifiers = max_ensemble_size

In [ ]:
pdict['prediction_mode'] = prediction_mode

In [ ]:
using_titanic = 0
using_gacrp = 0
using_elo = 1

### Load example dataset

In [ ]:
data_mode = 'kaggle'
# using_titanic = 0

In [ ]:
dataname = 'eurusd_1h'
input_source = np.load("data/data_%s.npy" % dataname)
to_predict = np.load("data/data_%s_targets.npy" % dataname)
to_predict = to_predict[3,:].reshape(-1)
input_source = input_source.T
is_orig = np.copy(input_source)
cp = int(0.8*len(input_source))
test_input_source = input_source[cp:, :]
test_to_predict = to_predict[cp:]
input_source = input_source[0:cp, :]
to_predict = to_predict[0:cp]
test_input_source = test_input_source.T
input_source = input_source.T

In [ ]:
winlen = 1
sliding_window_jump = 1
predict_time_ahead = 1
binary = 1

def prepare_data(input_source, to_predict, binary=1, cpp=0.75):

    x = []
    y = []

    ups = []
    downs = []
    mids = []

    for i in range(0, input_source.shape[1]-(winlen+100), sliding_window_jump):
        # form the input
        txs = input_source[:, i:i+winlen]#.astype(np.float64)
        #xs = scale(xs, axis=1)
        xs = txs.ravel()

        # for the output
        now = to_predict[i+winlen-1]# close
        future = to_predict[i+winlen+(predict_time_ahead-1)] # next close

        ys = future-now
        magn = abs(ys)

        if not binary:
            if magn < sep:
                mids.append( (xs, (np.array([1]))) )
            else:
                if ys < 0:
                    downs.append( (xs, (np.array([2]))) )
                else:
                    ups.append( (xs, (np.array([0]))) )
        else:
            if ys <= 0:
                downs.append( (xs, (np.array([1]))) )
            else:
                ups.append( (xs, (np.array([0]))) )

        #if (i%1000)==0:
        #    clear_output(wait=True)
        #    print('({}/{})'.format(i, input_source.shape[1]-winlen))

    #print('Ups: {0} Mids: {1} Downs: {2}'.format(len(ups), len(mids), len(downs)))
    #print('Separator: %3.5f' % sep)
    #print()

    if not binary:
        samplesize = min(len(ups), len(downs), len(mids))
        s1 = rnd.sample(ups, samplesize)
        s2 = rnd.sample(downs, samplesize)
        s3 = rnd.sample(mids, samplesize)
        a = s1 + s2 + s3
    else:
        samplesize = min(len(ups), len(downs))
        s1 = rnd.sample(ups, samplesize)
        s2 = rnd.sample(downs, samplesize)
        a = s1 + s2 
    rnd.shuffle(a)
    x = [x[0] for x in a]
    y = [x[1] for x in a]

    x = np.vstack(x)
    y = np.vstack(y)
    
    x.shape = (x.shape[0], -1)
    cutpoint = int(cpp * x.shape[0])
    x_train = x[0:cutpoint]
    x_test = x[cutpoint:]
    y_train = y[0:cutpoint]
    y_test = y[cutpoint:]
    
    return x_train, x_test, y_train.reshape(-1), y_test.reshape(-1)

In [ ]:
x_train, x_test, y_train, y_test = prepare_data(input_source, to_predict)

dx = vstack([x_train, x_test])
dy = hstack([y_train, y_test])

In [ ]:
x_val_real = dx

In [ ]:
dx = dx[0:nrows]
dy = dy[0:nrows]

In [ ]:
# dx[where(isnan(dx))] = 0

In [ ]:
dx.shape

In [ ]:
if parallel:
    ic = """
from aidevutil import *
from massimport import *
from project_common import *
from paramspace import *
    """

    if socket.gethostname() not in ['laptop', 'desktop']:
        idirs = ['/home/ubuntu', 
                 ]
        chd = ['/home/ubuntu/new/automl', 
               ]
    else:
        idirs = [ 
                 '/home/peter/code/projects']
        chd = [ 
               '/home/peter/code/work/automl']


    objs = {}
    objs['dx'] = dx
    objs['dy'] = dy
    objs['pdict'] = pdict
    objs['decode_classifier'] = decode_classifier
    objs['decode_classifier_mc'] = decode_classifier_mc
    objs['decode_classifier2'] = decode_classifier2
    objs['decode_regressor'] = decode_regressor
    objs['decode_preprocessor'] = decode_preprocessor
    objs['mappings'] = mappings
    objs['mappings_2nd_level'] = mappings_2nd_level
    objs['sk_classes'] = sk_classes
    objs['rg_classes'] = rg_classes
    objs['pc_classes'] = pc_classes
    objs['mc_classes'] = mc_classes
    objs['prediction_mode'] = prediction_mode
    objs = {**objs, **sk1_classes, **sk_classes, **rg_classes, **sndlev_classes, **pc_classes}

    rc, dview, lbview = init_ipyparallel(cluster_name='default', use_dill=False, imports_code = ic, 
                                         idirs = idirs, prep_code = None, chdirs = chd, objects = objs)

### Test all models first

In [ ]:
args = [(k,{},dx,dy,pdict) 
        for k in ar_mapping[prediction_mode].keys()]

In [ ]:
#%%pixie_debugger

if not parallel:
    rs = []
    for x in args:
        try:
            l = x[0], general_compute(x)
            print(l)
            rs.append(l)
        except Exception as ex:
            print(ex)
else:
    # evaluate all in parallel 
    rs = []
    pb = pbar.ProgressBar(max_value=len(args))
    for i, (r, x) in enumerate(zip(lbview.imap(general_compute, args, ordered=True), args)):
        pb.update(i)
        #print(pr, ms, sl, '%3.4f' % r)
        ms = x[0]
        rs.append((ms, r))
    pb.finish()

In [ ]:
if sort_mapping[prediction_mode] == 'min':
    srs = sorted(rs, key=lambda x: x[1], reverse=0)
else:
    srs = sorted(rs, key=lambda x: x[1], reverse=1)
srs = [x for x in srs if x[1] != -0.01 and not isnan(x[1])]
srs, len(srs)

In [ ]:
N=3
models = [x[0] for x in srs[0:N]]

In [ ]:
gc.collect()

In [ ]:
preps = list(pc_classes.keys())

In [ ]:
if not use_preps: preps = ['Raw']

In [ ]:
variance_d = 1

In [ ]:
hdx = dx[0:hpo_nrows]
hdy = dy[0:hpo_nrows]

In [ ]:
#fname = "/tmp/checkpoint_%s.pkl" % uuname()
#checkpoint_saver = CheckpointSaver(fname, compress=9)

def optimize_classifier(classifier, verbose=True):
    order = sorted(list(param_spaces[classifier].keys()))

    def testclf(args):
        ps = {}
        for a,o in zip(args,order):
            sp = param_spaces[classifier][o]['space']
            if isinstance(sp[0], int) and isinstance(sp[1], int):
                ps[o] = a
            elif isinstance(sp[0], float) and isinstance(sp[1], float):
                ps[o] = a
            else:
                ps[o] = param_spaces[classifier][o]['space'][a]
        return general_compute((classifier, ps, hdx, hdy, pdict))
    
    bl = []
    for o in order:
        sp = param_spaces[classifier][o]['space']
        if isinstance(sp[0], int) and isinstance(sp[1], int):
            bl.append((sp[0], sp[1]))
        elif isinstance(sp[0], float) and isinstance(sp[1], float):
            bl.append((sp[0], sp[1]))
        else:
            bl.append( (0, len(sp)-1) )

    res=None
    try:
        res = gp_minimize(testclf, bl, 
                          n_calls=50,
                          n_random_starts=10, 
                          #callback=[checkpoint_saver],
                          #noise=variance_data[classifier],
                          verbose=verbose)
    except KeyboardInterrupt:
        pass#res = load(fname)
    
    except Exception as ex:
        print(ex)
        
    a = list(zip(order,res.x))
    na = []
    for o, am in a:
        sp = param_spaces[classifier][o]['space']
        if isinstance(sp[0], int) and isinstance(sp[1], int):
            na.append( [o, am] )
        elif isinstance(sp[0], float) and isinstance(sp[1], float):
            na.append( [o, am] )
        else:
            na.append( [o, param_spaces[classifier][o]['space'][am]] )
        
    return dict(na)

In [ ]:
# pre-optimize the classifiers
if use_hpo_skopt:
    nsrs = []
    optimized = {}
    for c,sc in srs:
        if (c in param_spaces) and param_spaces[c]:
            d= optimize_classifier(c, verbose=1)
            nsc = [general_compute((c, d, dx, dy, pdict)) for _ in tqdm(range(variance_d))]
            nsrs.append((c,nsc))
            optimized[c] = d
            print(c,mean(nsc))
        else:
            nsrs.append((c,sc))
            optimized[c] = {}

    nsrs = sorted(nsrs, key=lambda x: mean(x[1]))

    # encode into model names
    models = [x[0]+' '+str(optimized[x[0]]) for x in nsrs]

In [ ]:
models

In [ ]:
def all_combs(models, n=2):
    ax = [models[:]]*n
    mx= [sorted(x) for x in list(product(*ax))]
    mx= [tuple(x) for x in mx if len(np.unique(x)) == len(x) ]
    mx= list(set(mx))
    return mx

In [ ]:
# HPO for all classifiers
if use_hpo:
    newmodels = []
    for k in models:
        if k in param_spaces and param_spaces[k]:
            ck= {k:v['space'] for k,v in param_spaces[k].items()}
            mk= list(product(*[v for k,v in sorted(ck.items())]))

            ks = [k for k,v in sorted(ck.items())]
            for m in mk:
                enc = str({k:v for k,v in zip(ks,m)})
                newmodels.append(k+' '+enc)
                #print(k+' '+enc)
        else:
            newmodels.append(k)
            #print(k)
            
    models = newmodels

In [ ]:
# HPO for all preprocessors
if use_hpo and use_preps:
    newpreps = []
    for k in preps:
        if k in param_spaces and param_spaces[k]:
            ck= {k:v['space'] for k,v in param_spaces[k].items()}
            mk= list(product(*[v for k,v in sorted(ck.items())]))

            ks = [k for k,v in sorted(ck.items())]
            for m in mk:
                enc = str({k:v for k,v in zip(ks,m)})
                newpreps.append(k+' '+enc)
                #print(k+' '+enc)
        else:
            newpreps.append(k)
            #print(k)

    preps = newpreps

In [ ]:
# lenght of models after HPO
len(models)

In [ ]:
# some of the models
rnd.shuffle(models)
models[0:5]

### Precomputing predictions

In [ ]:
# The preprocessed data 
dpx = {}
dpxv = {}
dpy = {}
dpyv = {}

In [ ]:
mdpx = {}
mdpxv = {}

In [ ]:
#%%eye
#%%pixie_debugger

if not parallel:
    
    pb = pbar.ProgressBar(max_value=len(preps)*num_trials + len(models)*len(preps)*num_trials )
    pl=0
    for trial in range(num_trials):
        data_x, x_val, data_y, y_val = datasplit(dx, dy, trial, num_trials=num_trials, test_size=test_size, mode=datasplit_mode)
        
        dpxt = {}
        dpxvt = {}
        
        # Preprocessing
        for i,n in enumerate(preps):
            c = decode_preprocessor(n,{})
            try:
                c.fit(data_x, data_y)
            except:
                try:
                    c.fit(data_x)
                except:
                    pass

            try:
                dpxt[n] = c.transform( data_x )
                dpxvt[n] = c.transform( x_val )
            except Exception as ex:
                print(n, ex)
            
            pb.update(pl)
            pl += 1

        dpx[trial] = dpxt
        dpxv[trial] = dpxvt 
        dpy[trial] = data_y
        dpyv[trial] = y_val
        
    for trial in range(num_trials):
        mdpx[trial] = {}
        mdpxv[trial] = {}
        
        for pn in preps:
            mdpx[trial][pn] = {}
            mdpxv[trial][pn] = {}
            
            for n in models:
                # fit classifier on data
                c = mappings[prediction_mode](n, prediction_mode)

                try:
                    c.fit(dpx[trial][pn], dpy[trial])

                    mdpx[trial][pn][n] = predictor(c, prediction_mode)( dpx[trial][pn] )
                    mdpxv[trial][pn][n] = predictor(c, prediction_mode)( dpxv[trial][pn] )
                    
                except Exception as ex:
                    print(n, ex)
                
                pb.update(pl)
                pl += 1
                
    pb.finish(end='\r')

In [ ]:
#%debug

In [ ]:
#mdpx[0]['Raw']['SVC'].shape

In [ ]:
# Parallel version of the above
def evala(args):
    trial, n, pn = args
    try:
        c = mappings[prediction_mode](n,{})
        c.fit(dpx[trial][pn], dpy[trial])
        p= predictor(c, prediction_mode)( dpx[trial][pn] )
        pv = predictor(c, prediction_mode)( dpxv[trial][pn] )
        return n, p, pv
    except:
        return n, None, None

def evalp(n):
    try:
        c = decode_preprocessor(n,{})
        c.fit(data_x, data_y)
    except:
        try:
            c.fit(data_x)
        except:
            pass
        
    try:
        p= c.transform( data_x ).astype(np.float).reshape(data_y.shape[0], -1)
        pv = c.transform( x_val ).astype(np.float).reshape(y_val.shape[0], -1)
        return n, p, pv
    except:
        return n, None, None

if parallel:
    for trial in range(num_trials):
        data_x, x_val, data_y, y_val = datasplit(dx, dy, trial, num_trials=num_trials, test_size=test_size, mode=datasplit_mode)
       # if prediction_mode != 'regression':
        #    data_y.shape = -1, 1
         #   y_val.shape = -1, 1
        
        dpxt = {}
        dpxvt = {}
        
        # update objects on cluster
        if use_preps:
            objs={}
            objs['data_x'] = data_x
            objs['data_y'] = data_y
            objs['x_val'] = x_val
            objs['y_val'] = y_val

            print('Sending objects to engines.. ', end='')
            kv = list(objs.items())
            for k, v in kv:
                if k != kv[-1][0]:
                    print(k, end=', ')
                else:
                    print(k, end='..')
                dview[k] = v
            dview.wait()
            print(' Done.')

            pb = pbar.ProgressBar(max_value=len(preps))
            for i, (n, p, pv) in enumerate(lbview.imap(evalp, preps)):
                dpxt[n] = p
                dpxvt[n] = pv
                pb.update(i)
            pb.finish(end='\r')
        else:
            n='Raw'
            dpxt[n] = data_x
            dpxvt[n] = x_val
        
        dpx[trial] = dpxt
        dpxv[trial] = dpxvt 
        dpy[trial] = data_y
        dpyv[trial] = y_val
        
    # update objects on cluster
    objs={}
    objs['dpx'] = dpx
    objs['dpxv'] = dpxv
    objs['dpy'] = dpy
    objs['dpyv'] = dpyv
    print('Sending objects to engines.. ', end='')
    kv = list(objs.items())
    for k, v in kv:
        if k != kv[-1][0]:
            print(k, end=', ')
        else:
            print(k, end='..')
        dview[k] = v
    dview.wait()
    print(' Done.')
    
    pb = pbar.ProgressBar(max_value=len(models)*len(preps)*num_trials)
    pl = 0
    for trial in range(num_trials):
        mdpx[trial] = {}
        mdpxv[trial] = {}
        for pn in preps:
            mdpx[trial][pn] = {}
            mdpxv[trial][pn] = {}
            args = [(trial, n, pn) for n in models]
            for i, (n, p, pv) in enumerate(lbview.imap(evala, args)):
                mdpx[trial][pn][n] = p 
                mdpxv[trial][pn][n] = pv 
                pb.update(pl)
                pl += 1
    pb.finish(end='\r')

### Stacking and voting functions 

In [ ]:
scores = {}
scores2 = {}

In [ ]:
# test non-stack
def test_regular(prep, stack, weights=None):
    accs=[]

    for trial in range(num_trials):
        # get predictions for x-val
        am = [mdpxv[trial][prep][n] for n in stack]
        a = voting_mixer(am, weights, prediction_mode)
        a = afterprocessing_mapping[prediction_mode](a)
        accs.append( metrics_mapping[prediction_mode](dpyv[trial], a) )
        
    return float(mean(accs))

def test_regular_par(args):
    prep, stack, = args
    try:
        weights=None
        if optimized_voting:
            ms=stack
            x0 = array([1/len(ms)]*len(ms))
            rzs = []
            for it in range(voting_opt_trials):
                magn = 0.8 if prediction_mode != 'regression' else 0.8
                r = x0 + np.random.uniform(low=-1 / len(ms) * magn, high=1 / len(ms) * magn, size=(len(ms),))
                rs = test_regular(prep, ms, weights=r)
                rzs.append((rs,r))
            ar = array([x[0] for x in rzs])
            if prediction_mode == 'regression':
                idx = where(ar==min(ar))[0][0]
            else:
                idx = where(ar==max(ar))[0][0]
            r, weights = ar[idx], rzs[idx][1]
        else:
            ms=stack
            r= test_regular(prep, ms)
        return r, prep, ms, weights
    except:
        return -1 if prediction_mode != 'regression' else 99999999, prep, stack, None

In [ ]:
#%%pixie_debugger 

def test_stack(prep, stack, slc):
    kp = []
    pts = None
    nnn= 'dummy'
    accs=[]

    for trial in range(num_trials):
        # get predictions for x
        if prediction_mode!='multiclass_classification':
            am = [mdpx[trial][prep][n].reshape(-1,1) for n in stack]
            if stack_original_level1:
                am.append( dpx[trial]['Raw'].reshape(-1,1) )
        else:
            am = [mdpx[trial][prep][n] for n in stack]
            if stack_original_level1:
                am.append( dpx[trial]['Raw'] )
        am = hstack(am)

        clf = mappings_2nd_level[prediction_mode](slc,prediction_mode)
        clf.fit(am, dpy[trial])
        # now predict on the validation set
        # get the individual classifier predictions, stack them, get the second level prediction
        if prediction_mode!='multiclass_classification':
            ps = [mdpxv[trial][prep][n].reshape(-1,1) for n in stack]
            if stack_original_level1:
                ps.append( dpxv[trial]['Raw'].reshape(-1,1) )
        else:
            ps = [mdpxv[trial][prep][n] for n in stack]
            if stack_original_level1:
                ps.append( dpxv[trial]['Raw'] )
        ps = hstack(ps)
        a = clf.predict(ps)
        a = a.astype(np.float)
        
        if pts is not None:
            ms = min(a.shape[0], pts.shape[0])
        else:
            ms = a.shape[0]
        if pts is not None: pts[0:ms,...] += a[0:ms, ...]
        else: pts = a
        
        a = afterprocessing_mapping[prediction_mode](a)
        accs.append( metrics_mapping[prediction_mode](dpyv[trial], a) )
        
    pts /= num_trials
    
    return float(mean(accs))

def test_stack_par(args):
    p, s, d = args
    try:
        return test_stack(p,s,d), p, s, d
    except Exception as ex:
        return -1 if prediction_mode != 'regression' else 99999999, p, s, d

### Exhaustive search

#### Stacking

In [ ]:
#%%pixie_debugger

# Test all combinations for stack lengths from 1 to 8 and all preprocessors
if exhaustive_search_on:
    comblens = 0
    mxs = []
    for n in range(min_ensemble_size, max_ensemble_size+1):
        mx = all_combs(models, n=n)
        comblens += len(mx)
        mxs.append(mx)
        
    if not parallel:
        thekeys = ar_mapping[prediction_mode].keys()
        allr = []
        pb = pbar.ProgressBar(max_value=len(list(thekeys))*len(preps)*comblens)
        pl=0
        for sl in thekeys:
            for pr in preps:
                for mx in mxs:
                    for ms in mx:
                        if 1:
                            r= test_stack(pr, ms, sl)
                        #except Exception as ex:
                        #    print(ex)
                        #    r = -1 if prediction_mode != 'regression' else 99999999
                        #print(pr, ms, sl, '%3.4f' % r)
                        allr.append((r, pr, ms, sl))
                        scores[ms] = r
                        pb.update(pl)
                        pl += 1
        pb.finish()

In [ ]:
#%debug

In [ ]:
# Parallel version of the above
if exhaustive_search_on:
    if parallel:
        # update objects on cluster
        objs={}
        objs['test_stack'] = test_stack
        objs['stack_original_level1'] = stack_original_level1
        objs['test_stack_par'] = test_stack_par
        objs['test_regular'] = test_regular
        objs['test_regular_par'] = test_regular_par
        objs['num_trials'] = num_trials
        objs['optimized_voting'] = optimized_voting
        objs['where'] = where
        objs['voting_opt_trials'] = voting_opt_trials
        objs['prediction_mode'] = prediction_mode
        objs['mdpx'] = mdpx
        objs['mdpxv'] = mdpxv
        objs['dpy'] = dpy
        objs['dpyv'] = dpyv
        objs['mean'] = mean
        objs['hstack'] = hstack
        objs['vstack'] = vstack
        objs['decode_classifier2'] = decode_classifier2
        #objs['regression_metric'] = regression_metric

        print('Sending objects to engines.. ', end='')
        kv = list(objs.items())
        for k, v in kv:
            if k != kv[-1][0]:
                print(k, end=', ')
            else:
                print(k, end='..')
            dview[k] = v
        dview.wait()
        print(' Done.')

        args = []
        allr = []
        thekeys = sndlev_classes.keys() if prediction_mode != 'regression' else rg_classes.keys()
        for sl in thekeys:
            for pr in preps:
                for n in range(min_ensemble_size, max_ensemble_size+1):
                    mx = all_combs(models, n=n)
                    for ms in mx:
                        args.append((pr, ms, sl))

        # evaluate all in parallel 
        pb = pbar.ProgressBar(max_value=len(args))
        for i, (r, pr, ms, sl) in enumerate(lbview.imap(test_stack_par, args)):
            pb.update(i)
            #print(pr, ms, sl, '%3.4f' % r)
            scores[ms] = r
            allr.append((r, pr, ms, sl))
        pb.finish()

In [ ]:
if exhaustive_search_on:
    if prediction_mode == 'regression':
        for x in sorted(allr, key=lambda x: x[0], reverse=False)[0:5]:
            print(x)
    else:
        for x in sorted(allr, key=lambda x: x[0], reverse=True)[0:5]:
            print(x)

#### Voting

In [ ]:
#%%eye

if exhaustive_search_on:
    if not parallel:
        allr2 = []
        for pr in preps:
            for n in range(min_ensemble_size, max_ensemble_size+1):
                mx = all_combs(models, n=n)
                for ms in mx:
                    weights=None
                    if optimized_voting:
                        x0 = array([1/len(ms)]*len(ms))
                        rzs = []
                        for it in range(voting_opt_trials):
                            r = x0 + np.random.uniform(low=-1 / len(ms) * 0.8, high=1 / len(ms) * 0.8, size=(len(ms),))
                            rs = test_regular(pr, ms, weights=r)
                            rzs.append((rs,r))
                        ar = array([x[0] for x in rzs])
                        if prediction_mode == 'regression':
                            idx = where(ar==min(ar))[0][0]
                        else:
                            idx = where(ar==max(ar))[0][0]
                        r, weights = ar[idx], rzs[idx][1]
                    else:
                        r= test_regular(pr, ms)
                    print(pr, ms,'%3.4f' % r)
                    scores2[ms] = r
                    allr2.append((r, pr, ms, weights))

In [ ]:
#%debug

In [ ]:
#%%eye

if exhaustive_search_on:
    if parallel:
        # update objects on cluster
        objs={}
        objs['test_stack'] = test_stack
        objs['test_regular'] = test_regular
        objs['test_regular_par'] = test_regular_par
        objs['num_trials'] = num_trials
        objs['mdpx'] = mdpx
        objs['mdpxv'] = mdpxv
        objs['dpy'] = dpy
        objs['dpyv'] = dpyv
        objs['mean'] = mean

        print('Sending objects to engines.. ', end='')
        kv = list(objs.items())
        for k, v in kv:
            if k != kv[-1][0]:
                print(k, end=', ')
            else:
                print(k, end='..')
            dview[k] = v
        dview.wait()
        print(' Done.')

        args = []
        allr2 = []
        for pr in preps:
            for n in range(min_ensemble_size, max_ensemble_size+1):
                mx = all_combs(models, n=n)
                for ms in mx:
                    args.append((pr,ms))

        # evaluate all in parallel 
        pb = pbar.ProgressBar(max_value=len(args))
        for i, (r, pr, ms, weights) in enumerate(lbview.imap(test_regular_par, args)):
            pb.update(i)
            #print(pr, ms,  '%3.4f' % r)
            scores2[ms] = r
            allr2.append((r, pr, ms, weights))
        pb.finish()

In [ ]:
if exhaustive_search_on:
    if prediction_mode == 'regression':
        for x in sorted(allr2, key=lambda x: x[0], reverse=False)[0:5]:
            print(x)
    else:
        for x in sorted(allr2, key=lambda x: x[0], reverse=True)[0:5]:
            print(x)

### GA 

In [ ]:
genetic4.models = models
genetic4.prep_models = preps

def evaluate_genome(asd):
    idx, traits, genome = asd
    wc, pp, slc = traits
    try:
        acc = test_stack(pp, wc, slc) # it does the many trials in test_stack
    except:
        acc = -1 
    #print(wc, acc)
    return idx, acc

def evaluate_genome_regular(asd):
    idx, traits, genome = asd
    wc, pp, slc = traits
    try:
        acc = test_regular(pp, wc) # it does the many trials in test_stack
    except:
        acc = -1
    #print(wc, acc)
    return idx, acc

def thetraits(x):
    return (x.GetGenomeTraits()['x'].cltypes, x.GetGenomeTraits()['x'].pp, x.GetGenomeTraits()['x'].slc )

def eval_args(x):
    return x.GetID(), thetraits(x), x

# called to display the new winner
def onimpr(pop, hof):
    try:
        print(thetraits(hof[-1]))
    except:
        pass

#### Stacking

In [ ]:
if genetic_search_on:
    run_name = uuname()

    g = NEAT.Genome(0, 1, 1, 1, 
                    0, NEAT.ActivationFunction.TANH,
                    NEAT.ActivationFunction.TANH, 0, params, 0)
    pop = NEAT.Population(g, params, True, 1.0, rnd.randint(0, 1000))

    #%%pixie_debugger

    evhist, hof = run_NEAT_loop(pop, num_generations=num_generations, 
                       num_evaluations=max_evaluations, 
                       parallel=0, rc=0, lbview = 0, generational_mode=generational_mode, 
                       evaluator=evaluate_genome, 
                       evaluator_args=eval_args, 
                       callback_on_improvement= onimpr
                  )

#### Voting

In [ ]:
if genetic_search_on:
    run_name = uuname()

    g = NEAT.Genome(0, 1, 1, 1, 
                    0, NEAT.ActivationFunction.TANH,
                    NEAT.ActivationFunction.TANH, 0, params, 0)
    pop = NEAT.Population(g, params, True, 1.0, rnd.randint(0, 1000))

    #%%pixie_debugger

    evhist2, hof2 = run_NEAT_loop(pop, num_generations=num_generations, 
                       num_evaluations=max_evaluations, 
                       parallel=0, rc=0, lbview = 0, generational_mode=generational_mode, 
                       evaluator=evaluate_genome_regular, 
                       evaluator_args=eval_args, 
                       callback_on_improvement= onimpr
                  )

### Tests

In [ ]:
# for tests, reduce num_trials to 1
onum_trials = num_trials
num_trials = 1
datasplit_mode='random'

In [ ]:
# get results from the vote retrained from scratch
def get_voting_preds(prep, stack, slc, dx, dy, weights=None):
    np.random.seed()
    preps = [prep]
    # The preprocessed data 
    dpx = {}
    dpxv = {}
    dpy = {}
    dpyv = {}
    
    mdpx = {}
    mdpxv = {}
    
    for trial in range(num_trials):
        data_x, x_val, data_y, y_val = datasplit(dx, dy, trial, num_trials=num_trials, test_size=test_size, mode=datasplit_mode)
        #if prediction_mode != 'regression':
          #  data_y.shape = -1, 1
           # y_val.shape = -1, 1
        dpxt = {}
        dpxvt = {}

        # Preprocessing
        for i,n in enumerate(preps):
            try:
                c = decode_preprocessor(n,{})
                c.fit(data_x, data_y)
            except:
                try:
                    c.fit(data_x)
                except:
                    pass
            try:
                dpxt[n] = c.transform( data_x )
                dpxvt[n] = c.transform( x_val )
            except:
                dpxt[n] = data_x
                dpxvt[n] = x_val
            
        dpx[trial] = dpxt
        dpxv[trial] = dpxvt 
        dpy[trial] = data_y
        dpyv[trial] = y_val

    for trial in range(num_trials):
        #mdpx[trial] = {}
        mdpxv[trial] = {}
        for pn in preps:
            #mdpx[trial][pn] = {}
            mdpxv[trial][pn] = {}
            for n in stack:
                # fit classifier on data
                c = mappings[prediction_mode](n,prediction_mode)
                try:
                    c.fit(dpx[trial][pn], dpy[trial])
                    #mdpx[trial][pn][n] = predictor(c, prediction_mode)( dpx[trial][pn] )
                    mdpxv[trial][pn][n] = predictor(c, prediction_mode)( dpxv[trial][pn] )
                except:
                    #mdpx[trial][pn][n] = dpx[trial][pn]
                    mdpxv[trial][pn][n] = dpxv[trial][pn]
    accs=[]
        
    for trial in range(num_trials):
        # get predictions for x
        am = [mdpxv[trial][prep][n] for n in stack]
        a = voting_mixer(am, weights, prediction_mode)        
        a = afterprocessing_mapping[prediction_mode](a)
            
        accs.append( metric_mapping[prediction_mode](dpyv[trial], a) )

    return mean(accs)

In [ ]:
# get results from the vote retrained from scratch
def get_voting_preds_new(prep, stack, slc, dx, dy, x_val_real, weights=None):
    np.random.seed()
    preps = [prep]
    # The preprocessed data 
    dpx = {}
    dpxv = {}
    dpy = {}
    dpyv = {}
    
    #mdpx = {}
    mdpxv = {}
    
    for trial in range(num_trials):
        data_x, x_val, data_y, y_val = dx, x_val_real, dy, None
        #if prediction_mode != 'regression':
        #    data_y.shape = -1, 1
        
        dpxt = {}
        dpxvt = {}

        if use_preps:
            # Preprocessing
            for i,n in enumerate(preps):
                try:
                    c = decode_preprocessor(n,{})
                    c.fit(data_x, data_y)
                except:
                    try:
                        c.fit(data_x)
                    except:
                        pass
                try:
                    dpxt[n] = c.transform( data_x )
                    dpxvt[n] = c.transform( x_val )
                except:
                    dpxt[n] = data_x
                    dpxvt[n] = x_val

        else:
            dpxt['Raw'] = data_x
            dpxvt['Raw'] = x_val
            
        dpx[trial] = dpxt
        dpxv[trial] = dpxvt 
        dpy[trial] = data_y
        dpyv[trial] = y_val

    for trial in range(num_trials):
        #mdpx[trial] = {}
        mdpxv[trial] = {}

        for pn in preps:
            #mdpx[trial][pn] = {}
            mdpxv[trial][pn] = {}

            for n in stack:
                # fit classifier on data
                c = mappings[prediction_mode](n,prediction_mode)
                try:
                    c.fit(dpx[trial][pn], dpy[trial])
                    #mdpx[trial][pn][n] = predictor(c, prediction_mode)( dpx[trial][pn] )
                    mdpxv[trial][pn][n] = predictor(c, prediction_mode)( dpxv[trial][pn] )
                except:
                    #mdpx[trial][pn][n] = dpx[trial][pn]
                    mdpxv[trial][pn][n] = dpxv[trial][pn]
    pds=[]
        
    for trial in range(num_trials):
        # get predictions for x
        am = [mdpxv[trial][prep][n] for n in stack]
        a = voting_mixer(am, weights, prediction_mode)
        a = afterprocessing_mapping[prediction_mode](a)
        pds.append(a) 

    return np.mean(array(pds), axis=0) 

In [ ]:
# get results from the stack retrained from scratch
def get_stack_preds(prep, stack, slc, dx, dy):
    preps = [prep]
    # The preprocessed data 
    dpx = {}
    dpxv = {}
    dpy = {}
    dpyv = {}
    
    mdpx = {}
    mdpxv = {}
    
    for trial in range(num_trials):
        data_x, x_val, data_y, y_val = datasplit(dx, dy, trial, num_trials=num_trials, test_size=test_size, mode=datasplit_mode)
        #if prediction_mode != 'regression':
           # data_y.shape = -1, 1
            #y_val.shape = -1, 1
        dpxt = {}
        dpxvt = {}
        
        # Preprocessing
        if use_preps:
            for i,n in enumerate(preps):
                try:
                    c = decode_preprocessor(n,{})
                    c.fit(data_x, data_y)
                except:
                    try:
                        c.fit(data_x)
                    except:
                        pass
                try:
                    dpxt[n] = c.transform( data_x )
                    dpxvt[n] = c.transform( x_val )
                except:
                    dpxt[n] = data_x
                    dpxvt[n] = x_val
                dpyt[n] = data_y
                dpyvt[n] = y_val
        else:
            dpxt['Raw'] = data_x
            dpxvt['Raw'] = x_val
            
        dpx[trial] = dpxt
        dpxv[trial] = dpxvt 
        dpy[trial] = data_y
        dpyv[trial] = y_val

    for trial in range(num_trials):
        mdpx[trial] = {}
        mdpxv[trial] = {}

        for pn in preps:
            mdpx[trial][pn] = {}
            mdpxv[trial][pn] = {}

            for n in stack:
                # fit classifier on data
                c = mappings[prediction_mode](n,prediction_mode)
                    
                try:
                    c.fit(dpx[trial][pn], dpy[trial])
                    mdpx[trial][pn][n] = predictor(c, prediction_mode)( dpx[trial][pn] )
                    mdpxv[trial][pn][n] = predictor(c, prediction_mode)( dpxv[trial][pn] )
                except:
                    mdpx[trial][pn][n] = dpx[trial][pn]
                    mdpxv[trial][pn][n] = dpxv[trial][pn]

        
    accs=[]
        
    for trial in range(num_trials):
        # get predictions for x
        if prediction_mode!='multiclass_classification':
            am = [mdpx[trial][prep][n].reshape(-1,1) for n in stack]
            if stack_original_level1:
                am.append( dpx[trial]['Raw'].reshape(-1,1) )
        else:
            am = [mdpx[trial][prep][n] for n in stack]
            if stack_original_level1:
                am.append( dpx[trial]['Raw'] )
        am = hstack(am)
        # train the second level classifier on the predictions 
        clf = mappings_2nd_level[prediction_mode](n,prediction_mode)
        clf.fit(am, dpy[trial])
        # now predict on the validation set
        # get the individual classifier predictions, stack them, get the second level prediction
        if prediction_mode!='multiclass_classification':
            ps = [mdpxv[trial][prep][n].reshape(-1,1) for n in stack]
            if stack_original_level1:
                ps.append( dpxv[trial]['Raw'].reshape(-1,1) )
        else:
            ps = [mdpxv[trial][prep][n] for n in stack]
            if stack_original_level1:
                ps.append( dpxv[trial]['Raw'] )
        ps = hstack(ps)
        a = clf.predict(ps)
        a = a.astype(np.float)
        a = afterprocessing_mapping[prediction_mode](a)
        accs.append( metric_mapping[prediction_mode](dpyv[trial], a) )

    return mean(accs)

In [ ]:
# get results from the stack retrained from scratch
def get_stack_preds_new(prep, stack, slc, dx, dy, x_val_real):
    preps = [prep]
    # The preprocessed data 
    dpx = {}
    dpxv = {}
    dpy = {}
    dpyv = {}
    
    mdpx = {}
    mdpxv = {}
    
    for trial in range(num_trials):
        data_x, x_val, data_y, y_val = dx, x_val_real, dy, None
        #if prediction_mode != 'regression':
        #    data_y.shape = -1, 1
        dpxt = {}
        dpxvt = {}
        # Preprocessing
        if use_preps:
            for i,n in enumerate(preps):
                try:
                    c = decode_preprocessor(n,{})
                    c.fit(data_x, data_y)
                except:
                    try:
                        c.fit(data_x)
                    except:
                        pass
                try:
                    dpxt[n] = c.transform( data_x )
                    dpxvt[n] = c.transform( x_val )
                except:
                    dpxt[n] = data_x
                    dpxvt[n] = x_val
        else:
            dpxt['Raw'] = data_x
            dpxvt['Raw'] = x_val

        dpx[trial] = dpxt
        dpxv[trial] = dpxvt 
        dpy[trial] = data_y
        dpyv[trial] = y_val

    for trial in range(num_trials):
        mdpx[trial] = {}
        mdpxv[trial] = {}

        for pn in preps:
            mdpx[trial][pn] = {}
            mdpxv[trial][pn] = {}

            for n in stack:
                # fit classifier on data
                c = mappings[prediction_mode](n,prediction_mode)
                    
                try:
                    c.fit(dpx[trial][pn], dpy[trial])
                    mdpx[trial][pn][n] = predictor( c, prediction_mode) (dpx[trial][pn] )
                    mdpxv[trial][pn][n] = predictor( c, prediction_mode) ( dpxv[trial][pn] )
                except:
                    mdpx[trial][pn][n] = dpx[trial][pn]
                    mdpxv[trial][pn][n] = dpxv[trial][pn]
        
    pds=[]
        
    for trial in range(num_trials):
        # get predictions for x
        if prediction_mode!='multiclass_classification':
            am = [mdpx[trial][prep][n].reshape(-1,1) for n in stack]
            if stack_original_level1:
                am.append( dpx[trial]['Raw'].reshape(-1,1) )
        else:
            am = [mdpx[trial][prep][n] for n in stack]
            if stack_original_level1:
                am.append( dpx[trial]['Raw'] )
        am = hstack(am)
        # train the second level classifier on the predictions
        clf = mappings_2nd_level[prediction_mode](n,prediction_mode)
        clf.fit(am, dpy[trial])
        # now predict on the validation set
        # get the individual classifier predictions, stack them, get the second level prediction
        if prediction_mode!='multiclass_classification':
            ps = [mdpxv[trial][prep][n].reshape(-1,1) for n in stack]
            if stack_original_level1:
                ps.append( dpxv[trial]['Raw'].reshape(-1,1) )
        else:
            ps = [mdpxv[trial][prep][n] for n in stack]
            if stack_original_level1:
                ps.append( dpxv[trial]['Raw'] )
        ps = hstack(ps)
        a = clf.predict(ps)
        a = a.astype(np.float).reshape(-1)
        a = afterprocessing_mapping[prediction_mode](a)
        pds.append(a)

    return np.mean(array(pds), axis=0)

#### Test stacking

#### Test voting

In [ ]:
# the shape of the data
print(dx.shape, dy.shape)

In [ ]:
# the shape of the data used to make Kaggle predictions
print(x_val_real.shape)

#### Predictions to Kaggle from stacking

In [ ]:
try:
    if prediction_mode == 'regression':
        p, m, sl = sorted(allr, key=lambda x: x[0], reverse=False)[0][-3:]
    else:
        p, m, sl = sorted(allr, key=lambda x: x[0], reverse=True)[0][-3:]
except Exception as ex:
    print(ex)
    ts = hof[-1].GetGenomeTraits()['x']
    m, p, sl = ts.cltypes, ts.pp, ts.slc    

In [ ]:
#%%pixie_debugger

# Compute predictions for Kaggle and bag them 
times=1
pcs = []
for i in range(times):
    print(i,'/',times,' '*10, end='\r')
    a = get_stack_preds_new(p, m, sl, dx, dy, x_val_real)
    pcs.append( a )

print(p, m, sl)
pc = mean(array(pcs), axis=0)

In [ ]:
if using_titanic:
    if titanic_version == 1:
        sub = pd.DataFrame()
        sub['PassengerId'] = p_id
        sub['Survived'] = pc.astype(int)
        sub.to_csv('Titanic_Submissions_Stack.csv', index=False)
    elif titanic_version == 2:
        probs = pc.astype(int)
        results = pd.DataFrame(probs, columns = ['Survived'])
        results['PassengerId'] = list(pd.read_csv('titanic_test.csv')['PassengerId'])
        (results[['PassengerId', 'Survived']]
            .to_csv('Titanic_Submissions_Stack.csv', index = False))

In [ ]:
if using_gacrp:
    from loaders.gacrp import df_test
    from sklearn.metrics import mean_squared_error

    def rmse(y_true, y_pred):
        return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

    def submit():
        test_matrix = x_val_real
        y_pred = pc#model.predict(test_matrix, verbose=1)
        df_test['PredictedLogRevenue'] = y_pred
        submit = df_test[['PredictedLogRevenue','fullVisitorId']].groupby('fullVisitorId').PredictedLogRevenue.sum().reset_index()
        submit.to_csv('submit_stacking.csv',index=False)

        test(y_pred)

    def test(predict):
        y_test = np.log1p(df_test['totals_transactionRevenue'])
        print(rmse(y_test,predict))
        
    submit()

In [ ]:
if using_elo:
    predictions = pc
    results = pd.DataFrame(predictions, columns = ['target'])
    if socket.gethostname() == 'desktop':
        results['card_id'] = list(pd.read_csv('/home/peter/code/work/automl/data/elo/sample_submission.csv')['card_id'])
    else:
        results['card_id'] = list(pd.read_csv('/home/ubuntu/new/automl/elo/sample_submission.csv')['card_id'])
    (results[['card_id', 'target']]
        .to_csv('ELO_automl_stack1_no_outliers.csv', index = False))
    print("created submissions file named ELO_automl_stack1.csv") 

#### Predictions to Kaggle from voting

In [ ]:
try:
    if prediction_mode == 'regression':
        p, m, ws = sorted(allr2, key=lambda x: x[0], reverse=False)[0][-3:]
    else:
        p, m, ws = sorted(allr2, key=lambda x: x[0], reverse=True)[0][-3:]
    sl = None
except:
    ts = hof2[-1].GetGenomeTraits()['x']
    m, p, sl = ts.cltypes, ts.pp, ts.slc
    ws=None

In [ ]:
# Compute predictions for Kaggle and bag them 
times=1
pcs = []
for i in range(times):
    print(i,'/',times,' '*10, end='\r')
    a = get_voting_preds_new(p, m, sl, dx, dy, x_val_real, weights=ws)
    pcs.append( a )

print(p, m, sl)
pc = mean(array(pcs), axis=0)

In [ ]:
if using_titanic:
    if titanic_version == 1:
        sub = pd.DataFrame()
        sub['PassengerId'] = p_id
        sub['Survived'] = np.round(pc).astype(int)
        sub.to_csv('Titanic_Submissions_Vote.csv', index=False)
    elif titanic_version == 2:
        probs = np.round(pc).astype(int)
        results = pd.DataFrame(probs, columns = ['Survived'])
        results['PassengerId'] = list(pd.read_csv('titanic_test.csv')['PassengerId'])
        (results[['PassengerId', 'Survived']]
            .to_csv('Titanic_Submissions_Vote.csv', index = False))

In [ ]:
if using_gacrp:
    from loaders.gacrp import df_test
    from sklearn.metrics import mean_squared_error

    def rmse(y_true, y_pred):
        return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

    def submit():
        test_matrix = x_val_real
        y_pred = pc#model.predict(test_matrix, verbose=1)
        df_test['PredictedLogRevenue'] = y_pred
        submit = df_test[['PredictedLogRevenue','fullVisitorId']].groupby('fullVisitorId').PredictedLogRevenue.sum().reset_index()
        submit.to_csv('submit_voting.csv',index=False)

        test(y_pred)

    def test(predict):
        y_test = np.log1p(df_test['totals_transactionRevenue'])
        print(rmse(y_test,predict))
        
    submit()

In [ ]:
if using_elo:
    predictions = pc
    results = pd.DataFrame(predictions, columns = ['target'])
    if socket.gethostname() == 'desktop':
        results['card_id'] = list(pd.read_csv('/home/peter/code/work/automl/data/elo/sample_submission.csv')['card_id'])
    else:
        results['card_id'] = list(pd.read_csv('/home/ubuntu/new/automl/elo/sample_submission.csv')['card_id'])
    (results[['card_id', 'target']]
        .to_csv('ELO_automl_vote1_no_outliers.csv', index = False))
    print("created submissions file named ELO_automl_vote1.csv") 